In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


## Preprocessing the data

In [8]:
#Drop irrelevant features
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


## since geography and gender are categorical variables we shall do encoding

In [82]:
label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.tail()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0
9999,792,0,28,4,130142.79,1,1,0,38190.78,0,1.0,0.0,0.0


In [12]:
## we can't labelEncode geography because it will give ranks to them so let's do oneHot encoding

from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])


In [19]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

## always remember to convert the encoded value into toarray() before concating

In [22]:
geo_encoded=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [23]:
geo_encoded

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


## now we have to drop the original geography columns and add these columns 

In [24]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded],axis=1)

In [25]:
data.tail()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0
9999,792,0,28,4,130142.79,1,1,0,38190.78,0,1.0,0.0,0.0


In [83]:
# save the pickle files
with open('label_encoder_gender.pkl','wb') as file:
  pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pkl','wb') as file:
  pickle.dump(onehot_encoder_geo,file)  

In [27]:
# divide the features into dependent and independent
X=data.drop('Exited',axis=1)
y=data['Exited']

# train test splitting
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# our next step is to normalise these columns
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
# save these files in pickle format
with open('scaler.pkl','wb') as file:
  pickle.dump(scaler,file)

# ANN implementation

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [30]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [34]:
(X_train.shape[1],) # this will be the number of inputs for input layer

(12,)

In [36]:
# building our ANN model
model = Sequential([
  Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #first hidden layer connected with input layer
  Dense(32,activation='relu'), #HL 2
  Dense(1,activation='sigmoid') # OUTPUT LAYER
]

)

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
# to add our own learning rate 
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [39]:
# compiling the model for forwardand backward propagation
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [40]:
# setup tensorboard
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [42]:
# setup earlystopping so that we don't need to overfit and stop at a certain epoch values where loss value decreasing has stoppped
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [43]:
#training the model
history = model.fit(
  X_train,y_train,validation_data=(X_test,y_test),epochs=100,
  callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 13s 21ms/step - loss: 0.3972 - accuracy: 0.8379 - val_loss: 0.3477 - val_accuracy: 0.8590
Epoch 2/100
250/250 [==============================] - 3s 12ms/step - loss: 0.3567 - accuracy: 0.8537 - val_loss: 0.3493 - val_accuracy: 0.8585
Epoch 3/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3534 - accuracy: 0.8576 - val_loss: 0.3517 - val_accuracy: 0.8595
Epoch 4/100
250/250 [==============================] - 3s 12ms/step - loss: 0.3462 - accuracy: 0.8571 - val_loss: 0.3400 - val_accuracy: 0.8585
Epoch 5/100
250/250 [==============================] - 4s 18ms/step - loss: 0.3419 - accuracy: 0.8574 - val_loss: 0.3430 - val_accuracy: 0.8640
Epoch 6/100
250/250 [==============================] - 4s 15ms/step - loss: 0.3379 - accuracy: 0.8602 - val_loss: 0.3406 - val_accuracy: 0.8600
Epoch 7/100
250/250 [==============================] - 3s 13ms/step - loss: 0.3351 - accuracy: 0.8621 - val_loss: 0.3463 - val_accurac

In [44]:
model.save('model.h5')

c:\Users\Saqlain\Desktop\ANN\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
# load tensor boaard extension
%load_ext tensorboard

In [48]:
%tensorboard --logdir logs/fit20240904-091503

Reusing TensorBoard on port 6006 (pid 14468), started 0:01:11 ago. (Use '!kill 14468' to kill it.)

# MAKING PREDICTONS

In [90]:
# loading the pickle files,trained model

from tensorflow.keras.models import load_model

model = load_model('model.h5')

with open('onehot_encoder_geo.pkl','rb') as file:
  onehot_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
  label_encoder_gender = pickle.load(file)  

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [91]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [92]:

oneHotEncoding = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_data = pd.DataFrame(oneHotEncoding,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

c:\Users\Saqlain\Desktop\ANN\venv\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [93]:
geo_data

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [94]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [100]:
## Encode categorical variables

input_df['Gender']=label_encoder_gender.fit_transform(input_df['Gender'])

In [101]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,60000,2,1,1,50000


In [102]:
# concatenation
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_data],axis=1)

In [103]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [104]:
input_scaled = scaler.transform(input_df)

In [105]:
input_scaled

array([[-0.53598516, -1.09499335,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [106]:
#prediction
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 4s 4s/step


array([[0.11428591]], dtype=float32)

In [108]:
prediction_probability = prediction[0,0]
prediction_probability

0.11428591

In [109]:
if prediction_probability>0.5:
  print('cuatomer likely to churn')
else:
  print('customer likely to not churn')  

customer likely to not churn
